In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, time, json
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm
import spacy
import pandas as pd

import sys
sys.path.append("../")
import os
from src.functional import free_gpu_cache

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from src.models import load_LM
import torch
from nnsight import LanguageModel
from src.utils import env_utils
from src.functional import free_gpu_cache

# model_key = "meta-llama/Meta-Llama-3-8B-Instruct"
model_key = "meta-llama/Meta-Llama-3-70B-Instruct"

lm = load_LM(
    model_key=model_key,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 30/30 [01:08<00:00,  2.28s/it]

loaded /home/local_arnab/Codes/00_MODEL/meta-llama/Meta-Llama-3-70B-Instruct | size: 36650.535 MB


In [4]:
from src.models import reset_forwards
from src.dataset import load_worldstate_dataset
from src.functional import predict_next_token 

dataset = load_worldstate_dataset()

In [5]:
#! why the experiment doesn't work if b is the correct answer?
reset_forwards(lm)
correct_ans_idx = 1
question, answer = dataset.__getitem__(22, tags=("a", "b"), correct_ans_idx=correct_ans_idx)
print(question)
print(answer)

Instruction: Keep track of people's knowledge defined in the story. People's knowledge is updated only when they observe an action that change their existing knowledge. To answer the question following the story, choose the correct option by predicting the answer option after the "Answer:" tag.

Story: Luka is a gardener working in a beautiful park in the middle of the city. Luka wants to water the plants in the park to ensure they stay healthy and vibrant. The sky is cloudy, and Luka assumes that it might rain soon. However, the wind picks up and blows the clouds away, revealing a bright and sunny sky. Luka notices the clouds clearing and the sun shining.

Question: Will it rain soon, or has the weather become sunny?
a) It will rain soon.
b) The weather has become sunny.
Choose one of the following:
Answer:
b


In [6]:
# question_1 = """Instruction: Keep track of people's knowledge defined in the story. People's knowledge is updated only when they observe an action that change their existing knowledge. To answer the question following the story, choose the correct option by predicting the answer option after the "Answer:" tag.

# Story: Noor is working as a barista at a busy coffee shop. Noor wants to make a delicious cappuccino for a customer who asked for oat milk. Noor grabs a milk pitcher and fills it with oat milk. A decorative mug falls from a shelf in the coffee shop. Noor does not see the decorative mug falling from the shelf.

# Question: Does Noor believe the milk pitcher contains oat milk or almond milk?
# a) Noor believes the milk pitcher contains almond milk.
# b) Noor believes the milk pitcher contains oat milk.
# Answer:"""

# answer = " b"

# question == question_1

In [7]:
from src.functional import predict_next_token
predict_next_token(
    lm = lm,
    input = question
)

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:435: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


[PredictedToken(token=' b', prob=0.9804607033729553, logit=19.890625, token_id=293),
 PredictedToken(token=' ', prob=0.0028192196041345596, logit=14.0390625, token_id=220),
 PredictedToken(token=' **', prob=0.002648411551490426, logit=13.9765625, token_id=3146),
 PredictedToken(token=' __', prob=0.002230186015367508, logit=13.8046875, token_id=1328),
 PredictedToken(token=' (', prob=0.0016834328416734934, logit=13.5234375, token_id=320)]

In [8]:
free_gpu_cache()

In [9]:
from src.models import prepare_input
from src.functional import find_token_range
input = prepare_input(
    prompts = question,
    tokenizer=lm,
    return_offsets_mapping=True,
)

option_1_ends = find_token_range(
    string=question,
    substring = ".\n",
    tokenizer=lm,
    occurrence=-2,
    offset_mapping=input["offset_mapping"][0],
)[1] - 1

option_2_ends = find_token_range(
    string=question,
    substring = ".\n",
    tokenizer=lm,
    occurrence=-1,
    offset_mapping=input["offset_mapping"][0],
)[1] - 1

input.pop("offset_mapping")
print(option_1_ends, lm.tokenizer.decode(input["input_ids"][0][option_1_ends]))
print(option_2_ends, lm.tokenizer.decode(input["input_ids"][0][option_2_ends]))

158 .

166 .



In [10]:
print(f"\"{lm.tokenizer.decode(input['input_ids'][0][:option_1_ends])}\"")

"<|begin_of_text|>Instruction: Keep track of people's knowledge defined in the story. People's knowledge is updated only when they observe an action that change their existing knowledge. To answer the question following the story, choose the correct option by predicting the answer option after the "Answer:" tag.

Story: Luka is a gardener working in a beautiful park in the middle of the city. Luka wants to water the plants in the park to ensure they stay healthy and vibrant. The sky is cloudy, and Luka assumes that it might rain soon. However, the wind picks up and blows the clouds away, revealing a bright and sunny sky. Luka notices the clouds clearing and the sun shining.

Question: Will it rain soon, or has the weather become sunny?
a) It will rain soon"


In [11]:
# from src.hooking.llama_attention import LlamaAttentionPatcher, KQV_patch
# import types
# import baukit
# from src.functional import interpret_logits


# free_gpu_cache()

# patch_range = list(range(35, 39))
# module_name_format = "model.layers.{}.self_attn"

# kqv_states = {layer_idx : {} for layer_idx in patch_range}
# kqv_spec = (
#     [(h_idx, option_1_ends) for h_idx in range(lm.config.num_attention_heads)] +
#     [(h_idx, option_2_ends) for h_idx in range(lm.config.num_attention_heads)]
# ) 

# reset_forwards(lm)
# for layer_idx in patch_range:
#     attn_module = baukit.get_module(
#         lm._model, module_name_format.format(layer_idx)
#     )
#     attn_module.forward = types.MethodType(
#         LlamaAttentionPatcher(
#             block_name=module_name_format.format(layer_idx),
#             save_kqv_for=kqv_spec,
#             kqv_states=kqv_states[layer_idx],
#         ), attn_module
#     )
    
# with torch.inference_mode():
#     with lm.trace(input, scan=False, validate=False) as trace:
#         logits = lm.output.logits[0, -1, :].save()

# free_gpu_cache()
# reset_forwards(lm)

# interpret_logits(
#     logits=logits,
#     tokenizer=lm
# )

In [12]:
free_gpu_cache()

In [13]:
# kqv_patches = {layer_idx : {} for layer_idx in patch_range}
# for layer_idx in kqv_states:
#     for h_idx in range(lm.config.num_attention_heads):
#         kqv_2 = kqv_states[layer_idx][(h_idx, option_2_ends)]
#         kqv_patches[layer_idx][(h_idx, option_1_ends)] = KQV_patch(
#             # k = kqv_2["key"],
#             v = kqv_2["value"],
#         )

#         kqv_1 = kqv_states[layer_idx][(h_idx, option_1_ends)]
#         kqv_patches[layer_idx][(h_idx, option_2_ends)] = KQV_patch(
#             # k = kqv_1["key"],
#             v = kqv_1["value"],
#         )
    

# reset_forwards(lm)
# for layer_idx in patch_range:
#     attn_module = baukit.get_module(
#         lm._model, module_name_format.format(layer_idx)
#     )
#     attn_module.forward = types.MethodType(
#         LlamaAttentionPatcher(
#             block_name=module_name_format.format(layer_idx),
#             kqv_patches=kqv_patches[layer_idx],
#         ), attn_module
#     )
# with torch.inference_mode():
#     with lm.trace(input, scan=False, validate=False) as trace:
#         logits = lm.output.logits[0, -1, :].save()

# reset_forwards(lm)
# free_gpu_cache()

# interpret_logits(
#     logits=logits,
#     tokenizer=lm
# )

In [18]:
from src.functional import get_module_nnsight, interpret_logits
from src.utils.typing import TokenizerOutput

@torch.inference_mode()
def cache_kqv_states(
    input: TokenizerOutput,
    lm: LanguageModel,
    patch_range: list[int],
    attn_module_name_format: str = "model.layers.{}.self_attn",
) -> dict[int, dict[str, torch.Tensor]]:
    kqv_states = {layer_idx : {} for layer_idx in patch_range}
    with lm.trace(input, scan=False, validate=False) as trace:
        for layer_idx in patch_range:
            module = get_module_nnsight(lm, attn_module_name_format.format(layer_idx))
            kqv_states[layer_idx] = {
                "q" : module.q_proj.output.clone().save(),
                "k" : module.k_proj.output.clone().save(),
                "v" : module.v_proj.output.clone().save(),
            }
    return kqv_states

patch_range = list(range(33, 40))
kqv_states = cache_kqv_states(input, lm, patch_range)

In [19]:
from src.utils.typing import PredictedToken
from typing import Literal

@torch.inference_mode()
def check_kqv_patching_effect(
    input: TokenizerOutput,
    lm: LanguageModel,
    swap_positions: tuple[int, int],
    kqv_states: dict[int, dict[str, torch.Tensor]],
    patch_type: list[Literal["k", "q", "v"]] = ["k"],
    attn_module_name_format: str = "model.layers.{}.self_attn",
    interested_tokens: list[int] = [],
) -> tuple[list[PredictedToken], dict]:
    option_1_ends, option_2_ends = swap_positions
    with lm.trace(input, scan=False, validate=False) as trace:
        for layer_idx in kqv_states:
            module = get_module_nnsight(lm, attn_module_name_format.format(layer_idx))
            
            if "q" in patch_type:
                module.q_proj.output[:, option_1_ends, :] = kqv_states[layer_idx]["q"][:, option_2_ends, :]
                module.q_proj.output[:, option_2_ends, :] = kqv_states[layer_idx]["q"][:, option_1_ends, :]

            if "k" in patch_type:
                module.k_proj.output[:, option_1_ends, :] = kqv_states[layer_idx]["k"][:, option_2_ends, :]
                module.k_proj.output[:, option_2_ends, :] = kqv_states[layer_idx]["k"][:, option_1_ends, :]

            if "v" in patch_type:
                module.v_proj.output[:, option_1_ends, :] = kqv_states[layer_idx]["v"][:, option_2_ends, :]
                module.v_proj.output[:, option_2_ends, :] = kqv_states[layer_idx]["v"][:, option_1_ends, :]
        
        logits = lm.output.logits[0, -1, :].save()
    
    predicted_tokens = interpret_logits(tokenizer = lm, logits = logits)
    
    for i in range(len(interested_tokens)):
        t = interested_tokens[i]
        if isinstance(t, str):
            interested_tokens[i] = lm.tokenizer.encode(t)[-1]
    rank_tokens = logits.argsort(descending=True).tolist()
    probs = torch.nn.functional.softmax(logits, dim=-1)
    track_interested_tokens = {
        t : {
            "rank": rank_tokens.index(t) + 1,
            "prob": PredictedToken(
                token = lm.tokenizer.decode(t),
                prob = probs[t].item(),
                logit = logits[t].item(),
                token_id = t,
            ),
        } for t in interested_tokens
    }

    return predicted_tokens, track_interested_tokens

check_kqv_patching_effect(
    input=input,
    lm=lm,
    swap_positions=(option_1_ends, option_2_ends),
    kqv_states=kqv_states,
    patch_type=["k"],
    interested_tokens=[" a", " b"]
)

([PredictedToken(token=' a', prob=0.9598780870437622, logit=19.046875, token_id=264),
  PredictedToken(token=' b', prob=0.00897938385605812, logit=14.375, token_id=293),
  PredictedToken(token=' The', prob=0.0053618354722857475, logit=13.859375, token_id=578),
  PredictedToken(token=' ', prob=0.004806318320333958, logit=13.75, token_id=220),
  PredictedToken(token=' (', prob=0.0034081898629665375, logit=13.40625, token_id=320)],
 {264: {'rank': 1,
   'prob': PredictedToken(token=' a', prob=0.9598780870437622, logit=19.046875, token_id=264)},
  293: {'rank': 2,
   'prob': PredictedToken(token=' b', prob=0.00897938385605812, logit=14.375, token_id=293)}})

In [22]:
import random

limit = 10
tag_options = [
    ("a", "b"),
    ("x", "y"), 
    ("p", "q"),
]
patch_range = list(range(35, 40))

patch_effects = []
limit = limit if limit < len(dataset) else len(dataset)


for idx in range(limit):
    option_labels = random.choice(tag_options)
    correct_ans_idx = random.choice([0, 1])
    
    question, answer = dataset.__getitem__(idx, tags=option_labels, correct_ans_idx=correct_ans_idx)
    clean_prediction = predict_next_token(lm=lm, input=question)[0]
    print(f"{idx=} | {answer=} | {clean_prediction=}")
    if clean_prediction.token.strip() != answer.strip():
        print(f"{idx=} model prediction is wrong, expected {answer.strip()} but got {clean_prediction} ... skipping")
        continue

    wrong_ans = option_labels[1 - correct_ans_idx]
    input = prepare_input(
        prompts = question,
        tokenizer=lm,
        return_offsets_mapping=True,
    )

    swap_positions = [
        (
            find_token_range(
                string=question,
                substring = ".\n",
                tokenizer=lm,
                occurrence=pos,
                offset_mapping=input["offset_mapping"][0],
            )[1] - 1
        ) for pos in [-2, -1]
    ]
    print(f"{swap_positions=}")

    kqv_states = cache_kqv_states(input=input, lm=lm, patch_range=patch_range)
    predicted_tokens, track_wrong_ans = check_kqv_patching_effect(
        input = input, 
        lm = lm, 
        swap_positions = swap_positions,
        kqv_states = kqv_states,
        patch_type=["k"],
        interested_tokens=[f" {wrong_ans}"]
    )
    patch_prediction = predicted_tokens[0]
    print(f"{patch_prediction=}")
    track_wrong_ans = list(track_wrong_ans.values())[0]
    print(track_wrong_ans)
    patch_effects.append(track_wrong_ans)
    print()

idx=0 | answer='p' | clean_prediction=PredictedToken(token=' p', prob=0.9829173684120178, logit=20.078125, token_id=281)
swap_positions=[167, 176]
patch_prediction=PredictedToken(token=' q', prob=0.8689221143722534, logit=19.0, token_id=2874)
{'rank': 1, 'prob': PredictedToken(token=' q', prob=0.8689221143722534, logit=19.0, token_id=2874)}

idx=1 | answer='y' | clean_prediction=PredictedToken(token=' y', prob=0.9763201475143433, logit=19.5625, token_id=379)
swap_positions=[177, 189]
patch_prediction=PredictedToken(token=' y', prob=0.9680306315422058, logit=19.578125, token_id=379)
{'rank': 2, 'prob': PredictedToken(token=' x', prob=0.009639687836170197, logit=14.96875, token_id=865)}

idx=2 | answer='p' | clean_prediction=PredictedToken(token=' p', prob=0.9817273616790771, logit=20.28125, token_id=281)
swap_positions=[146, 154]
patch_prediction=PredictedToken(token=' q', prob=0.8963739275932312, logit=19.1875, token_id=2874)
{'rank': 1, 'prob': PredictedToken(token=' q', prob=0.896373

In [24]:
efficacy = sum([1 for p in patch_effects if p["rank"] == 1]) / len(patch_effects)
print(f"Patch efficacy: {efficacy}")

Patch efficacy: 0.7
